Modules

In [2]:
import pandas as pd
from rapidfuzz import fuzz, process  # Fuzzy matching for flexibility
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
from nltk.corpus import wordnet
from nltk import pos_tag


In [3]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity


In [4]:
# Download all necessary NLTK resources
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger_eng')  # NEW - required for POS tagging

# Initialize lemmatizer and stopword list
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words("english"))


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\nick_\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\nick_\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nick_\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\nick_\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


Important Functions

In [5]:
# Helper to convert NLTK POS to WordNet POS
def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN  # fallback

# Updated preprocess_text function
def preprocess_text(text):
    if not isinstance(text, str):
        return ""

    words = text.lower().split()
    words = [word for word in words if word not in stop_words]
    tagged_words = pos_tag(words)

    lemmatized_words = []
    for word, pos in tagged_words:
        wn_pos = get_wordnet_pos(pos)
        lemma = lemmatizer.lemmatize(word, wn_pos)
        if lemma == word and wn_pos != wordnet.VERB:
            # Retry as verb if unchanged
            lemma = lemmatizer.lemmatize(word, pos='v')
        lemmatized_words.append(lemma)

    return " ".join(lemmatized_words)

def remove_bracketed_words(text):
    if not isinstance(text, str):
        return ""
    
    # Remove all content inside brackets (including brackets)
    cleaned_text = re.sub(r'\s*\([^)]*\)', '', text).strip()
    
    return cleaned_text


# Function to find synonyms using WordNet
def get_synonyms(word):
    synonyms = set()
    for syn in wordnet.synsets(word):
        for lemma in syn.lemmas():
            synonyms.add(lemma.name().replace("_", " "))  # Convert underscores to spaces
    return list(synonyms)



Flattening the ESCO Data Set

In [6]:
# Load the dataset
file_path = "../data/ESCO skill taxonomy dataset.csv"
df = pd.read_csv(file_path)

# Columns containing skills
skill_columns = [
    "Essential Skills (Skill)",
    "Essential Skills (Knowledge)",
    "Optional Skills (Skill)",
    "Optional Skills (Knowledge)"
]

# Ensure missing values are handled
df[skill_columns] = df[skill_columns].fillna("")

# Split comma-separated skills into lists
for col in skill_columns:
    df[col] = df[col].apply(lambda x: x.split(", ") if isinstance(x, str) else [])

# Explode each skill column separately and merge back
esco_taxonomy_df = df.copy()
for col in skill_columns:
    esco_taxonomy_df = esco_taxonomy_df.explode(col)

# Rename columns for clarity
esco_taxonomy_df = esco_taxonomy_df.rename(columns={
    "Occupation Title": "Job Role",
    "ESCO Code": "ESCO Code",
    "Description": "Job Description",
    "Alternative Labels": "Alternative Titles",
    "Essential Skills (Skill)": "Essential Skill",
    "Essential Skills (Knowledge)": "Essential Knowledge",
    "Optional Skills (Skill)": "Optional Skill",
    "Optional Skills (Knowledge)": "Optional Knowledge"
})



In [7]:
columns_to_preprocess = [
    "Essential Skill",
    "Essential Knowledge",
    "Optional Skill",
    "Optional Knowledge"
]

for col in columns_to_preprocess:
    unique_values = esco_taxonomy_df[col].dropna().unique()
    processed_map = {val: preprocess_text(val) for val in unique_values}
    esco_taxonomy_df[col] = esco_taxonomy_df[col].map(processed_map)

esco_taxonomy_df["Essential Skill"] = esco_taxonomy_df["Essential Skill"].apply(remove_bracketed_words)
esco_taxonomy_df["Essential Knowledge"] = esco_taxonomy_df["Essential Knowledge"].apply(remove_bracketed_words)
esco_taxonomy_df["Optional Skill"] = esco_taxonomy_df["Optional Skill"].apply(remove_bracketed_words)
esco_taxonomy_df["Optional Knowledge"] = esco_taxonomy_df["Optional Knowledge"].apply(remove_bracketed_words)


Matching skills to jobs

In [8]:
# Create a set of all valid skills for fuzzy matching
all_skills = set(
    esco_taxonomy_df["Essential Skill"].dropna().tolist() +
    esco_taxonomy_df["Essential Knowledge"].dropna().tolist() +
    esco_taxonomy_df["Optional Skill"].dropna().tolist() +
    esco_taxonomy_df["Optional Knowledge"].dropna().tolist()
)

essential_skills = set(    
    esco_taxonomy_df["Essential Skill"].dropna().tolist() +
    esco_taxonomy_df["Essential Knowledge"].dropna().tolist()
    )

optional_skills = set(    
    esco_taxonomy_df["Optional Skill"].dropna().tolist() +
    esco_taxonomy_df["Optional Knowledge"].dropna().tolist()
)

In [9]:
file_path = "../data/tools_grouped.csv"

# Load Technology Tool Mapping
tech_tools_df = pd.read_csv(file_path)  # File containing tool-category mappings
tool_to_category = {}

for _, row in tech_tools_df.iterrows():
    raw_category = row["Technology Tool"]
    raw_tools = row["Technology Tool Example"].split(", ")

    # Preprocess the category name once
    category = preprocess_text(raw_category)

    for tool in raw_tools:
        preprocessed_tool = preprocess_text(tool)
        tool_to_category[preprocessed_tool] = category


This code performs intelligent matching between user-entered skills and job roles in the ESCO taxonomy, using a combination of fuzzy string matching and tool-category mapping. The map_skills_to_matched_tools() function takes an individual skill and tries to match it either directly to a known tool name or to a broader tool category using fuzzy matching (with a configurable similarity threshold). It returns the best-matching tool category based on score comparisons. The main function, match_jobs(), processes a list of user-input skills by first cleaning them and then attempting to match each one — prioritizing exact matches for short single-character skills (like "R"), followed by fuzzy matches against tools and the ESCO skills dataset. Top matches are selected based on similarity scores, and low-confidence or ambiguous matches are filtered out. Once the matching is done, the function aggregates all matched skills and compares them against job roles in the ESCO taxonomy by constructing a set of all skills (essential and optional) associated with each job. It counts how many matched user skills appear in each job's skill set, filters out roles with fewer than two matches, and returns a ranked list of job roles based on how well they align with the user’s skills. The output includes the number of overlapping skills per job and displays the best-matching roles for easier interpretation.


In [55]:

def map_skills_to_matched_tools(skill, threshold=85):
    """
    Matches user-entered skill to either a tool or tool category based on fuzzy score.
    Returns the best-matched tool category.
    """
    tool_names = list(tool_to_category.keys())
    tool_categories = list(set(tool_to_category.values()))
    skill_lower = preprocess_text(skill)

    # Fuzzy match with tool names
    match_result_tool = process.extractOne(skill_lower, tool_names, scorer=fuzz.WRatio)
    tool_match, tool_score = match_result_tool[:2] if match_result_tool else (None, 0)
    tool_category = tool_to_category.get(tool_match) if tool_match else None

    # Fuzzy match with category names
    match_result_category = process.extractOne(skill_lower, tool_categories, scorer=fuzz.WRatio)
    category_match, category_score = match_result_category[:2] if match_result_category else (None, 0)

    # Decide based on best score
    if tool_score >= threshold and (tool_score > category_score):
        print(f"🔁 '{skill}' matched to tool '{tool_match}' → category '{tool_category}' (score: {tool_score})")
        return tool_category
    elif category_score >= threshold:
        print(f"🔁 '{skill}' matched directly to category '{category_match}' (score: {category_score})")
        return category_match

    # No good match
    return None



# Main Matching Function
def match_skills_esco_fuzzy(user_skills, threshold): 
    """
    Matches user-entered skills with job roles using NLP + fuzzy matching.
    Filters jobs that have at least 2 matched skills.
    """

    matched_skills = set()

    # Preprocess user input skills
    user_skills = [preprocess_text(skill) for skill in user_skills]

    # Ensure all_skills only contains valid skills
    cleaned_skills = set(skill for skill in all_skills if isinstance(skill, str) and skill.strip())

    # Perform fuzzy matching for each user skill
    for skill in user_skills:
        print(f"\n🔍 Matching User Skill: {skill}")

        # ✅ Exact match logic for single-character user skills (e.g., "r")
        if len(skill.strip()) == 1:
            exact_matches = [s for s in cleaned_skills if s.strip().lower() == skill.strip().lower()]
            if exact_matches:
                matched_skills.add(exact_matches[0])
                print(f"✅ Exact Match for Short Skill: {exact_matches[0]}")
            else:
                print("❌ No exact match found for short skill.")
            continue  # Skip fuzzy matching for this skill

        # Tool match
        tool_raw = map_skills_to_matched_tools(skill, threshold=threshold)
        tool_match = preprocess_text(tool_raw) if tool_raw else None
        tool_match_result = process.extractOne(tool_match, cleaned_skills, scorer=fuzz.WRatio) if tool_match else None
        tool_best, tool_score = tool_match_result[:2] if tool_match_result else (None, 0)
        print(f"Found Tool Match (Tech Tool): {tool_best} with score: {tool_score}")
        # Fuzzy match top 5
        top_matches = process.extract(skill, cleaned_skills, scorer=fuzz.WRatio, limit=5)
        print("Top Matches:")
        for i, (match, score, _) in enumerate(top_matches, start=1):
            print(f"  {i}. {match} ({round(score, 1)})")

        # ✅ Match all top skills with the same max score above threshold
        if tool_best and tool_score >= threshold and tool_score > top_matches[0][1]:
            print(f"✅ Selected Tool Match (Tech Tool): {tool_best}")
            matched_skills.add(tool_best)
        elif top_matches:
            max_score = top_matches[0][1]
            if max_score >= threshold:
                for match, score, _ in top_matches:
                    if score == max_score:
                        # ✅ Prevent single-letter *matched* skills unless perfect match
                        if len(match.strip()) == 1 and score < 100:
                            print(f"⚠️ Skipped short matched skill '{match}' (score: {score}) — not an exact match.")
                            continue
                        matched_skills.add(match)
                        print(f"✅ Selected Tie Match: {match} ({round(score, 1)})")
            elif tool_best and tool_score >= threshold:
                matched_skills.add(tool_best)
                print(f"✅ Selected Tool Match (Tech Tool): {tool_best}")
        else:
            print("❌ No good match found.")

    return matched_skills



In [56]:
def match_skills_job(matched_skills):

    matched_skills = set(skill.lower().strip() for skill in matched_skills)

    if not matched_skills:
        print("\n🚫 No valid matches found.")
        return pd.DataFrame(columns=["Job Role", "Matched Skills Count"])


    # Step 2: Create 'All Skills' column per job
    esco_taxonomy_df["All Skills"] = (
        esco_taxonomy_df["Essential Skill"].fillna("").astype(str) + ", " +
        esco_taxonomy_df["Essential Knowledge"].fillna("").astype(str) + ", " +
        esco_taxonomy_df["Optional Skill"].fillna("").astype(str) + ", " +
        esco_taxonomy_df["Optional Knowledge"].fillna("").astype(str)
    )

    # Step 3: Group by Job and create a set of lowercase skill strings
    job_skills_df = (
        esco_taxonomy_df.groupby(["Job Role", "Job Description"])["All Skills"]
        .apply(lambda skills: set(s.strip().lower() for line in skills for s in line.split(",") if s.strip()))
        .reset_index(name="All Skills Set")
    )

    # Step 4: Count how many matched skills appear in each job
    job_skills_df["Matched Skills Count"] = job_skills_df["All Skills Set"].apply(
        lambda skills: len(matched_skills.intersection(skills))
    )

    # Step 5: Filter and sort
    matched_jobs = job_skills_df[job_skills_df["Matched Skills Count"] > 1].copy()
    matched_jobs = matched_jobs.sort_values(by="Matched Skills Count", ascending=False)

    # Step 6: Display results
    print("\n📋 Matched Jobs (At Least 2 Skills Matched):")
    print("-" * 58)
    print("| {:<30} | {:<20} |".format("Job Role", "Matched Skills Count"))
    print("-" * 58)
    for _, row in matched_jobs.iterrows():
        print("| {:<30} | {:<20} |".format(row["Job Role"], row["Matched Skills Count"]))
    print("-" * 58)

    return matched_jobs[["Job Role", "Matched Skills Count"]]


In [62]:
cybersecurity_analyst_skills = ["Network security",
        "SIEM (e.g., Splunk)",
        "Threat analysis",
        "Vulnerability assessment",
        "Firewalls and intrusion detection systems",
        "Risk assessment",
        "Incident response",
        "Encryption techniques",
        "Security policies",
        "Linux"]
found_skills = match_skills_esco_fuzzy(cybersecurity_analyst_skills, threshold=85)

match_skills_job(found_skills)


🔍 Matching User Skill: network security
🔁 'network security' matched directly to category 'network security virtual private network vpn software' (score: 90.0)
Found Tool Match (Tech Tool): develop virtual game engine with score: 85.5
Top Matches:
  1. ict network security risk (90.0)
  2. identify ict security risk (85.5)
  3. network management system tool (85.5)
  4. advise strengthen security (85.5)
  5. analyse network bandwidth requirement (85.5)
✅ Selected Tie Match: ict network security risk (90.0)

🔍 Matching User Skill: siem (e.g., splunk)
🔁 'siem (e.g., splunk)' matched to tool 'splunk siem' → category 'network monitor software' (score: 85.5)
Found Tool Match (Tech Tool): author software with score: 85.5
Top Matches:
  1. apl (60.0)
  2. ansible (54.0)
  3. swift (51.4)
  4. mysql (51.3)
  5. implement strategic plan (46.5)
✅ Selected Tool Match (Tech Tool): author software

🔍 Matching User Skill: threat analysis
🔁 'threat analysis' matched directly to category 'motion anal

,Job Role,Matched Skills Count
20,ICT resilience manager,6
23,ICT security manager,6
63,ethical hacker,5
9,ICT disaster recovery analyst,4
22,ICT security consultant,4
38,chief ICT security officer,3
6,ICT capacity planner,3
60,embedded systems security engineer,3
34,IT auditor,3
56,digital forensics expert,3


This code defines a two-step pipeline for matching user-provided skills to the ESCO skills taxonomy using sentence embeddings and cosine similarity. The prepare_esco_embeddings() function takes a raw set of ESCO skills, cleans it by removing empty strings and extra whitespace, and then uses the pre-trained Sentence-BERT model (all-MiniLM-L6-v2) to encode the cleaned skills into numerical embeddings. These embeddings represent the semantic meaning of each skill in vector space. The match_user_skills() function then preprocesses the user's input skills (e.g., lowercasing, stripping whitespace), encodes them using the same SBERT model, and compares them to the ESCO embeddings using cosine similarity. For each user skill, it identifies the top matching ESCO skill(s) and returns the results in a DataFrame showing the original user skill, the closest matched ESCO skill, and their similarity score. This approach enables semantic matching even when the exact phrasing of skills differs.



In [58]:

def prepare_skill_embeddings(raw_skills):
    """
    Cleans and encodes a set of ESCO skills using SBERT.
    Returns the cleaned skill list, model, and embeddings.
    """
    model = SentenceTransformer('all-MiniLM-L6-v2')

    # Case 1: Dictionary input
    if isinstance(raw_skills, dict):
        items = [
            k.strip() for k in raw_skills.keys()
            if isinstance(k, str) and k.strip() != ''
        ]
        embeddings = model.encode(items)
        return items, model, embeddings, raw_skills

    # Case 2: List or set input
    elif isinstance(raw_skills, (list, set)):
        items = [
            s.strip() for s in raw_skills
            if isinstance(s, str) and s.strip() != ''
        ]
        embeddings = model.encode(items)
        return items, model, embeddings, None

    else:
        raise TypeError("Input must be a list, set, or dictionary.")


def match_user_skills(user_skills, skills, embeddings, model, threshold=0.7, top_k=1):
    """
    Matches user-entered skills to a set of reference skills using cosine similarity.
    Returns a nicely printed DataFrame of best matches and the matched skills list.
    """
    matched_skills = []
    user_skills = [preprocess_text(skill) for skill in user_skills]
    user_embeddings = model.encode(user_skills)

    records = []

    for i, user_emb in enumerate(user_embeddings):
        sims = cosine_similarity([user_emb], embeddings)[0]
        top_indices = sims.argsort()[::-1][:top_k]

        best_match_idx = top_indices[0]
        best_score = sims[best_match_idx]

        if best_score >= threshold:
            matched_skill = skills[best_match_idx]
            matched_skills.append(matched_skill)
        else:
            matched_skill = "No Match"

        records.append({
            "User Skill": user_skills[i],
            "Matched Skill": matched_skill,
            "Cosine Similarity": round(best_score, 3)
        })

    df = pd.DataFrame(records)

    # ✅ Pretty print
    print("\n📋 Skill Matching Results")
    print("-" * 60)
    print(df.to_markdown(index=False))
    print("-" * 60)

    return matched_skills


In [59]:
esco_skills, esco_model, esco_embeddings, _ = prepare_skill_embeddings(all_skills)

anzsco_skills, anzsco_model, anzsco_embeddings, anzsco_lookup  = prepare_skill_embeddings(tool_to_category)


In [65]:
skills = [
        "SQL",
        "Oracle or MySQL",
        "Backup and recovery",
        "Performance tuning",
        "Database security",
        "Stored procedures",
        "Indexing",
        "Data migration",
        "Monitoring tools",
        "Scripting (e.g., Bash, Python)"
    ]
esco_matches = match_user_skills(skills, esco_skills, esco_embeddings, esco_model, 0.70)

anzsco_tool_matches = match_user_skills(skills, anzsco_skills, anzsco_embeddings, anzsco_model, 0.70)

tools_found_list = []
for tool in anzsco_tool_matches:
    tools_found_list.append(tool_to_category[tool])

esco_tool_matches = match_user_skills(tools_found_list, esco_skills, esco_embeddings, esco_model, 0.70)

combined_matches = esco_tool_matches + esco_matches

match_skills_job(combined_matches)



📋 Skill Matching Results
------------------------------------------------------------
| User Skill                  | Matched Skill              |   Cosine Similarity |
|:----------------------------|:---------------------------|--------------------:|
| sql                         | sql server                 |               0.736 |
| oracle mysql                | mysql                      |               0.714 |
| backup recovery             | perform backup             |               0.731 |
| performance tune            | No Match                   |               0.506 |
| database security           | maintain database security |               0.91  |
| store procedure             | No Match                   |               0.521 |
| index                       | No Match                   |               0.368 |
| data migration              | migrate exist data         |               0.747 |
| monitor tool                | No Match                   |               0.655 |


,Job Role,Matched Skills Count
47,data centre operator,5
52,database administrator,5
51,data warehouse designer,4
53,database designer,4
54,database developer,4
75,system configurator,4
55,database integrator,3
21,ICT security administrator,2
25,ICT system administrator,2
